<p>
<font size='5' face='Georgia, Arial'>IIC2115 - Programación como herramienta para la ingeniería</font><br>
</p>

# Web Scraping

En este capítulo haremos una introducción a ***Web Scraping***. Este es el proceso de extraer información de sitios web. El scraping de datos se enfoca en transformar el contenido no estructurado de un sitio web (usualmente HTML) en datos estructurados los que pueden ser almacenados en una base datos, en una hoja de cálculo o en el mismo código.

La forma en que los datos son extraídos de un sitio web es similar a la utilizada por los bots de búsqueda - la navegación web humana es simulada utilizando programas (bots) los que extraen (scrape) los datos de un sitio web.






## Tipos de páginas Web

A la hora de hacer scrapping en páginas web, es importante distinguir cómo está construida y alojada la información. Esta puede estar contenida directamente en el html (estática) o puede ser generada por alguna incrustación dinámica dentro del html como un javaScript (dinámica).

La información estática es de más facil acceso ya se encuentra contenida directamente en el html y es en la que profundizaremos.

Si necesita trabajar con páginas web deinámicas es recomendable investigar la librería `Selenium` de Python. Esta permite simular un usuario que interactúa en una página web donde se permite la recolección de datos.

## Aplicación con Beatiful Soup

Para el caso de realizar Web Scraping en páginas que són estáticas se recomienda el uso de la librería `Beatiful Soup`. Con esta librería desarrollaremos el siguiente ejemplo. Suponga que se requiere desarrollar una aplicación que permita conocer el valor de la UTM en todo momento. Para conocer dicho valor podemos basarnos en el que publica la web "prensa digital" [aqui](https://www.prensadigital.cl/valor-utm-unidad-tributaria-mensual.html).

### PASO 1: Conocer y familiarizarse con la web

Es fuertemente recomendable que utilicen el navegador Google Chrome para abrir la url. Una vez abierta, debe seleccionar el valor del dolar y con click secundario selecciones "inspeccionar elemento". Ahora podrán ver el código de la pagina web y familiarizarse con ella.

### PASO 2: Descargar el html

Ahora utilizaremos la librería urllib para simular un navegador y descargar el código fuente de la página:

In [ ]:
import urllib.request as net
import ssl

In [ ]:
class WebDownloader:

    def __init__(self, link):
        self.user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        self.url =  link


    def getHtmlAsString(self):
        headers = {'User-Agent':self.user_agent}
        request= net.Request(self.url,None,headers)
        gcontext = ssl.create_default_context() # Using create_default_context for better practice and to avoid deprecation warnings
        response = net.urlopen(request,context=gcontext)
        return response.read().decode('utf-8')

wd = WebDownloader('https://www.prensadigital.cl/valor-utm-unidad-tributaria-mensual.html')
sourceCode = wd.getHtmlAsString()
print(sourceCode)

### PASO 3: Buscar el valor solicitado

Al revisar el código fuente, es posible notar que el valor de la UTM se encuentra dentro del archivo que acabamos de descargar. En este caso basta con "jugar" con el string descargado para poder obtener el valor buscado.

Pero existe una forma un poco más directa. Podemos inspeccionar el código fuente en el browser y seleccionar los elementos del sitio web que almacenan el valor buscado. En este caso, vemos que el valor de la UTM se presenta en múltiples partes del sitio. En este caso, utilizaremos específicamente el que se almacena en la tabla inferior, que se encuentra representado en el código de la siguiente manera:

`<td>61.769</td>`

esta dentro del nodo "td", en este caso podemos acceder a todos los nodos con ese nombre:

In [ ]:
import bs4

In [ ]:
info = []
soup = bs4.BeautifulSoup(sourceCode)

for node in soup.find_all('td'):
    info.append(str(u''.join(node.find_all(string=True)).encode('utf-8'))[2:-1])

print(info)

En este caso vemos como apuntaba a la tabla de la web.

### PASO 4: Mostrar la información

In [ ]:
print("El valor actual de la UTM es: {}".format(info[1]))

Funciona!

En general Web Scraping es a medida. Es decir, deben desarrollarse distintas herramientas en función de las necesidades. Documentarse correctamente es fundamental.

## Web Scraping con pandas

También podemos realizar un ejemplo básico de web scraping utilizando requests, BeautifulSoup y pandas. Primero descargamos el contenido HTML de una página de Wikipedia y usamos BeautifulSoup para encontrar la tabla que contiene los datos de comunas de Chile. Luego, pandas.read_html se encarga de parsear automáticamente la tabla HTML y convertirla en un DataFrame. Esta función internamente utiliza lxml o html5lib para interpretar el contenido estructurado de la tabla y extraerlo como datos tabulares, lo que simplifica enormemente el proceso de extracción de datos desde sitios web.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

url = "https://es.wikipedia.org/wiki/Anexo:Comunas_de_Chile"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Encontramos la tabla que contiene las comunas
table = soup.find_all("table", {"class": "wikitable"})[0]
df = pd.read_html(StringIO(str(table)))[0]
df.head()

Este tipo de extracción es útil cuando se necesita recopilar información actualizada desde la web que aún no está disponible en formato descargable o API. Una vez obtenida la tabla como DataFrame, se pueden inspeccionar y depurar los datos, realizar análisis, visualizaciones o integrarla con otros conjuntos de datos. El web scraping permite así automatizar la recolección de información desde sitios públicos, facilitando tareas que de otro modo requerirían recopilación manual.


## Acceso a datos vía API (REST)

Además de obtener información desde el HTML de una página web, otra forma eficiente y profesional de obtener datos es mediante una API (Application Programming Interface).

Una **API** es un conjunto de reglas y especificaciones que permiten que dos aplicaciones se comuniquen entre sí. Muchas plataformas públicas y privadas exponen APIs para que podamos consultar sus datos de forma controlada, estructurada y actualizada.

A diferencia del web scraping (que puede romperse si cambia el HTML), las APIs ofrecen estabilidad, formato consistente (generalmente JSON) y documentación oficial.

### Tipos de APIs comunes
- **REST APIs**: accedidas vía URLs y usando métodos HTTP como GET, POST.
- **GraphQL APIs**: consultas flexibles en un solo endpoint.
- **APIs privadas**: requieren autenticación más estricta, a veces vía OAuth.    

### ¿Cómo funcionan las APIs REST?
Para acceder a una API REST, normalmente se hace una solicitud HTTP a una URL específica (llamada endpoint). Las solicitudes más comunes son:

- `GET`: para obtener datos.
- `POST`: para enviar datos o crear nuevos recursos.
- `PUT` y `PATCH`: para actualizar información.
- `DELETE`: para eliminar recursos.

Estas solicitudes se pueden hacer desde herramientas como `curl`, `Postman`, o directamente desde Python usando la librería `requests`.

### Formato de respuesta
La mayoría de las APIs REST devuelven los datos en formato JSON (JavaScript Object Notation), que es ligero, fácil de leer y de procesar en Python.

Ejemplo de una respuesta JSON:
{
  "comuna": "Providencia",
  "poblacion": 150000,
  "superficie_km2": 14.4
}

### Autenticación y uso de API Keys
Muchas APIs requieren autenticación para controlar el acceso. El método más simple es el uso de una API key (clave privada) que se incluye en la URL o en los headers. Algunas APIs públicas no requieren autenticación, pero otras pueden usar tokens temporales, autenticación básica o flujos más complejos como OAuth 2.0.

### Extracción de datos con una API.
En este ejemplo usaremos una REST API pública: **OpenWeatherMap**, para obtener el clima actual de una ciudad. Para obtener la `API KEY` propia, es necesario registrarse gratis [aquí](https://openweathermap.org/price#:~:text=Learn%20more-,Free%20Access,-Current%20weather%20and).

No es recomendado dejar las `API KEYS` en el notebook, para evitarlo se recomienda utilizar un archivo `.env` y la librería `dotenv` para leerlo.

In [ ]:
import os
import requests
from getpass import getpass
import pandas as pd

In [ ]:
os.environ["API_KEY"] = getpass("Ingresa tu API KEY de OpenWeatherMap: ")

In [ ]:
API_KEY = os.getenv("API_KEY")
ciudad = "Santiago"
url = f"http://api.openweathermap.org/data/2.5/weather?q={ciudad}&appid={API_KEY}&units=metric&lang=es"

respuesta = requests.get(url)
datos = respuesta.json()
datos


Podemos inspeccionar el objeto JSON devuelto por la API y extraer información relevante.
    

In [ ]:

info_clima = {
    "Ciudad": datos["name"],
    "Temperatura (°C)": datos["main"]["temp"],
    "Sensación térmica (°C)": datos["main"]["feels_like"],
    "Temperatura mínima (°C)": datos["main"]["temp_min"],
    "Temperatura máxima (°C)": datos["main"]["temp_max"],
    "Humedad (%)": datos["main"]["humidity"],
    "Presión (hPa)": datos["main"]["pressure"],
    "Velocidad del viento (m/s)": datos["wind"]["speed"],
    "Dirección del viento (°)": datos["wind"]["deg"],
    "Descripción del clima": datos["weather"][0]["description"],
    "Nubosidad (%)": datos["clouds"]["all"],
    "Visibilidad (m)": datos["visibility"]
}

pd.DataFrame([info_clima]).transpose()


### ¿Por qué usar APIs?

- Son más confiables que el scraping. Basta que cambien aspectos mínimos de una página web para dejar obsoleto un scrapper.
- Datos actualizados directamente desde la fuente.
- El formato suele estar pensado para el consumo de datos.
- Permiten acceder a grandes volúmenes de datos de forma controlada, escalable y organizada.
- Muchas permiten filtros, autenticación y personalización de respuestas.

### Buenas prácticas:
- Nunca subir tus credenciales o API keys a repositorios públicos. Utilizar un archivo `.env` y la librería `dotenv` para leerlas.
- Consultar la documentación oficial de cada API.
- Limitar la frecuencia de tus consultas (respetar el "rate limit").

    